## Predicting Education Level

## Introduction

People go into the job industry with varying circumstances regarding the way they were raised, taught, and born. At some point in our lives we are made to think about the question, "What should I do when I grow up?" Most of the time the answer lands on finding a path to make a living out of. But as young people who have not fully experienced what being a part of society is like yet, we wish to find a path that leads us to the most success without pouring too many resources into achieving the needed qualifications, and the biggest obstacle to that is education. The decision to pursue a higher education is both time and money-consuming, and there is no taking back choice once it has been made. That is why our Data Science project question is: "Can we predict the highest level of education someone has received based on their salary, years of experience, and age?"

The dataset we are using contains 7 columns reading left to right: `Education`, `Experience`, `Location`, `Job_Title`, `Age`, `Gender`, `Salary`, where each row represents an individual worker.

- `Education`: four categories, each representing the highest level of education received: High School, Bachelor, Master, PhD
- `Experience`: years of total work experience earned in their listed job
- `Location`: location of work split into three categories: Urban, Suburban, Rural
- `Job_Title`: four general types of job titles: Manager, Director, Analyst, Engineer
- `Age`: age of the worker
- `Gender`: Female or Male
- `Salary`: salary of the worker  

## Preliminary exploratory data analysis

- Demonstrate that the dataset can be read from the web into R
- Clean and wrangle your data into a tidy format
- Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data.
- Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your ld this lead to?

## Methods

- Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
- Describe at least one way that you will visualize the results

## Expected outcomes and significance

We expect to find that higher numbers in all predictor fields (salary, years of work experience, and age) predict a higher level of education.

We wish to use our findings to help someone determine what education level, years of experience, and age they should be in order to reach a certain salary range. These results may impact the way people choose to allocate the years of their life, and hopefully help them reach a decision in a less ambiguous way.

This analysis could lead to future questions about whether or not people should be spending their time and money on higher education in order to reach their desired financial position.